**Nome:** Camily Milsoni Rodrigues

Importação do modelo de linguagem "DialoGPT-medium" da Microsoft e seu tokenizer, além do "torch" para manipulação de tensores:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Implementação de um loop simples, que é repetido 5 vezes, para um chatbot interativo em que o usuário fornece entradas textuais e o modelo responde com base no histórico da conversa:

In [ ]:
for step in range(5):
  new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
  bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
  chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
  print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:Hello!


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


DialoGPT: Hello! :D
>> User:How are you doing?
DialoGPT: I'm doing well! How are you?
>> User:I'm fine.
DialoGPT: That's good.
>> User:Have a good day.
DialoGPT: You too!
>> User:Bye.
DialoGPT: Bye! :D


Importação da biblioteca "Pandas":

In [ ]:
import pandas as pd

Criação de um dicionário de dados que contém 2 listas, associando os números dos pedidos com os seus respectivos status:

In [ ]:
dados_pedidos = {
  "numero_pedido": ["12345", "67890", "11121", "22232"],
  "status": ["Shipped", "Processing", "Delivered", "Cancelled"]
}

Conversão do dicionário de dados criado anteriormente em um DataFrame da biblioteca "Pandas":

In [ ]:
df_status_pedidos = pd.DataFrame(dados_pedidos)

Visualização do DataFrame para verificar se ele foi criado corretamente:

In [ ]:
df_status_pedidos

,numero_pedido,status
0,12345,Shipped
1,67890,Processing
2,11121,Delivered
3,22232,Cancelled


Definição de uma função que verifica o status de um pedido com base em seu número:

In [ ]:
def verificar_status_pedido(numero_pedido):
  try:
    status = df_status_pedidos[df_status_pedidos["numero_pedido"] == numero_pedido]["status"].iloc[0]
    return f"The status of your order {numero_pedido} is: {status}"
  except:
    return "Order number not found. Please check and try again."

Criação de uma lista de palavras-chave que será usada na identificação de intenções relacionadas à consulta do status de pedidos:

In [ ]:
palavras_chave_status = ["order", "order status", "status of my order", "check my order", "track my order", "order update"]

Implementação de um chatbot interativo que verifica a intenção do usuário a cada entrada. Se o usuário digitar algo relacionado ao status de pedidos, o bot solicita o número do pedido e usa a função "verificar_status_pedido" para retornar o status. Caso contrário, o chatbot utiliza um modelo de linguagem para gerar uma resposta baseada no histórico da conversa. O chatbot também permite que o usuário encerre a conversa digitando comandos como "exit", "quit" ou "stop".

In [ ]:
ids_historico_chat = None

while True:
  input_usuario = input("You: ")
  if input_usuario.lower() in ['exit', 'quit', 'stop']:
    print("Bot: Goodbye!")
    break

  if any(keyword in input_usuario.lower() for keyword in palavras_chave_status):
    numero_pedido = input('Could you please enter your order number?')
    resposta = verificar_status_pedido(numero_pedido)
  else:
    novo_usuario_input_ids = tokenizer.encode(input_usuario + tokenizer.eos_token, return_tensors='pt')

    if ids_historico_chat is not None:
      bot_input_ids = torch.cat([ids_historico_chat, novo_usuario_input_ids], dim=-1)

    else:
      bot_input_ids = novo_usuario_input_ids

    ids_historico_chat = model.generate(
      bot_input_ids,
      max_length=1000,
      pad_token_id=tokenizer.eos_token_id
    )
    resposta = tokenizer.decode(ids_historico_chat[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

  print(f'Bot: {resposta}')

You: Hello
Bot: Hello! :D
You: How are you doing?
Bot: I'm doing well! How are you?
You: I'm great.
Bot: That's good!
You: I would like to check the status of my order.
Could you please enter your order number?12345
Bot: The status of your order 12345 is: Shipped
You: Thanks a lot.
Bot: No problem. :D
You: stop
Bot: Goodbye!


In [ ]:
pip install gradio

Criando a interface para o ChatBot:

In [ ]:
import gradio as gr

dados = {
    "order_number": [12345, 67890, 13579, 24680],
    "status": ["Shipped", "Processing", "Delivered", "Canceled"]
}
df_dados = pd.DataFrame(dados)

espera_num_pedido = False

def verificar_pedido(order_number):
    try:
        status = df_dados[df_dados["order_number"] == int(order_number)]["status"].iloc[0]
        return f"The status of your order {order_number} is: {status}"
    except (IndexError, ValueError):
        return "Order number not found. Please check and try again."

palavras_chave = ["order", "order status", "status of my order", "check my order", "track my order", "order update"]

def chatbot_respondendo(user_input, chat_history=None):
    global espera_num_pedido

    if espera_num_pedido:
        response = verificar_pedido(user_input)
        espera_num_pedido = False
        return response, chat_history

    if any(keyword in user_input.lower() for keyword in palavras_chave):
        espera_num_pedido = True
        return "Could you please enter your order number?", chat_history

    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    if chat_history is not None:
        bot_input_ids = torch.cat([chat_history, new_user_input_ids], dim=-1)
    else:
        bot_input_ids = new_user_input_ids

    chat_history = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(chat_history[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response, chat_history

def chatbot_interface(user_input, history):
    response, chat_history = chatbot_respondendo(user_input, history)
    return response, chat_history

app = gr.Interface(fn=chatbot_interface,
                     inputs=["text", "state"],
                     outputs=["text", "state"],
                     title="ChatBot para pedidos",
                     description="Ask about your order status or chat with the bot.")

app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://21c15fc15cb251025b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
